In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
output_csv = os.getenv("INPUT_CSV")
df = pd.read_csv(output_csv)

In [ ]:
video_ids = df['video_id'].unique()

X_list = []
y_list = []

for vid in video_ids:
    video_df = df[df['video_id'] == vid].sort_values('frame')
    
    features = video_df.drop(columns=['video_id', 'frame', 'label']).values
    
    if features.shape[0] == 90:
        X_list.append(features)
        y_list.append(video_df['label'].iloc[0]) 

X = np.array(X_list)
y = np.array(y_list)
print(X.shape, y.shape)

(3840, 90, 26) (3840,)


In [5]:
num_samples, timesteps, num_features = X.shape
X_reshaped = X.reshape(-1, num_features)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_reshaped)

X_scaled = X_scaled.reshape(num_samples, timesteps, num_features)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

model = Sequential([
    Masking(mask_value=0., input_shape=(timesteps, num_features)), 
    LSTM(64, return_sequences=False),
    Dense(32, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')  # klasyfikacja wieloklasowa
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=8,
    verbose=1
)

c:\Users\skrza\Documents\GitHub\ENGINEERING-WORK\.venv\lib\site-packages\keras\src\layers\core\masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 90, 26)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        23,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,607 (100.03 KB)

 Trainable params: 25,607 (100.03 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.2410 - loss: 1.8851 - val_accuracy: 0.2699 - val_loss: 1.8805
Epoch 2/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.2652 - loss: 1.8612 - val_accuracy: 0.3106 - val_loss: 1.5523
Epoch 3/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.3323 - loss: 1.5603 - val_accuracy: 0.3886 - val_loss: 1.5160
Epoch 4/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.3905 - loss: 1.4807 - val_accuracy: 0.4732 - val_loss: 1.1912
Epoch 5/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.4775 - loss: 1.2073 - val_accuracy: 0.5480 - val_loss: 1.0739
Epoch 6/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.6042 - loss: 0.9333 - val_accuracy: 0.6764 - val_loss: 0.8797
Epoch 7/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.6781 - loss: 0.7780 - val_accuracy: 0.7545 - val_loss: 0.6257
Epoch 8/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.7269 - loss: 0.6571 - val_accu

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {acc:.4f}")